# Métodos espectrais de alta ordem na resolução de equações diferenciais
<center> <img src="./Figuras/ime.png"  width="200"> </center>
Orientador:
## Prof. Dr. Nelson Mukgayar Kuhl [IME - USP]

Coorientador:
## Dr. Paulo José Saiz Jabardo [CTMETRO - IPT]


# História

* proposto por Blinova em 1944
* implementado por Silberman em 1954
* praticamente abandonado na década de 60 
* Formalizado matematicamente por Gottlieb e Orszag em 1977
* e vem crescendo até hoje

Uso:
 * usado por metereologistas para modelos globais de tempo
 * estudo de  dinâmica dos fluidos computacional [CFD]
     * resolver problemas de equações diferenciais de Sturm-Liouville
 * parte da pesquisa cresce lado a lado com o avanço tecnológico

# A monografia
 Neste trabalho, irei realizar algumas das técnicas do método espectral para a resolução de equações diferenciais em 1D (uma dimensão) apresentados por G. Karniadakis e S. Sherwin [**Spectral/hp Element Methods
for CFD**].


# Linguagem Julia

<center> <img src="./Figuras/julia.png"  width="100"> </center>

* Aspectos positivos :
     * características de linguagem de alto nível:
         * polimorfismo
         * tipagem dinâmica
         * fácil implementação
     * desempenho comparável a linguagens de baixo nível como (C e fortran)
     * comunidade científica em crescimento
     * utiliza algumas bibliotecas já conhecidas da linguagem python 


# Linguagem Julia

<center> <img src="./Figuras/julia.png"  width="100"> </center>

* Aspectos negativos :
    * linguagem muito nova (Ver. 0.5)
        * constante mudança de funções da biblioteca base 
        * falta de algumas bibliotecas expecíficas (que estão sendo implementadas)
    


# Tópicos
1. método dos resíduos ponderados
2. Métodos espectral (interpolação, diferenciação, integração)
3. Fenômeno Runge
4. elementos finitos
5. resultados
6. Conclusões

# Método dos resíduos ponderados

Dado $u(x)$ a solução da equação diferencial:
 \begin{align}
     L(u) &= \frac{\partial^2 u}{\partial x^2} + \lambda u + f = 0 \\
    L(u) &= 0
 \end{align}

# Método dos resíduos ponderados
 Aproximando $u$ por $u^\delta$ obtemos um erro $R(u^\delta)$ associado ao operador $L(\bullet)$:
 \begin{align}
 	u^\delta(x) &=  \sum^{N_{dof}}_{i = 1} u(x_i) \Phi(x)\\
    L(u^\delta) &= R(u^\delta)
\end{align}

# Método dos resíduos ponderados

Nosso problema está na minimização desse erro $R(u^\delta)$. Para isso, escolhemos uma função teste $v_j$ onde $j = 1,2,\dots, N_{dof}$, tal que seu produto interno com o erro seja o menor possível.
\begin{equation}
    <v_j,R(u^\delta)> = \int_{\Omega} v_j(x)\ R(x) \partial x,\ \forall j = 1,2,3,\dots,N_{dof}\\
\end{equation}

# Qual $V_j$ escolher ?

# Método dos resíduos ponderados

\begin{array}{|c|c|}
\hline Função\ teste & Método  \\\hline
   v_j(x)=\ \delta(x\ - \ x_j)  & \text{Colocação}\\\hline
   v_j\ =\ \frac{\partial R}{\partial \hat{u}_j} & \text{Min. Quadrados} \\\hline
   v_j(x) \equiv \phi_j\ \text{e}\ v_j(\Omega_d) = 0 & \text{Galerkin} \\\hline   
\end{array}

# Método de Galerkin

Dado a equação diferencial de Poison:
 \begin{equation} 
 L(u) \equiv \frac{\partial^2 u}{\partial x^2} + f = 0
 \end{equation}
Definida em, $\Omega = \{x| -1 < x < 1\}$
 \begin{equation}
 u(-1)=g_D\ ,\ \frac{\partial u(1)}{\partial x} = g_N
 \end{equation}
 Onde $g_D$ e $g_N$, são constantes, e são chamadas de condição de contorno de *D*irichlet e *N*eumman. Essas condições junto da equação , chamamos essa combinação como formulação *Forte* do problema.

# Formulação fraca

 Como a função teste $v$ é zero na fronteira de *Dirichlet*, sabemos que $v(-1) = 0$. Assim, aplicando a condição de fronteira de *Neumman*, $\frac{\partial u(1)}{\partial x} = g_N$, simplificamos a equação, temos:
\begin{equation}
 (v,L(u))=\int^1_{-1} v\left ( \frac{\partial^2u}{\partial x^2} + f \right )\partial x = 0
\end{equation}


# Formulação fraca

Podemos ver que essa equação pode ser reescrita usando integração por partes obtemos:

 \begin{align}
& \int^{1}_{-1} v \frac{\partial^2 u}{\partial x^2} \partial x = \left [ v\frac{\partial u}{\partial x}    \right ]^{1}_{-1} - \int^{1}_{-1} \frac{\partial v}{\partial x}  \frac{\partial u}{\partial x}  \partial x \\
 \end{align}


# Formulação fraca

\begin{equation}
 \int^{1}_{-1} \frac{\partial v}{\partial x}  \frac{\partial u}{\partial x}  \partial x =  v(1)g_N + \int^{1}_{-1}  v f\ \partial x  
\end{equation}

Vemos que nessa última etapa, a condição de fronteira de *Neumman* é naturalmente inclusa na formulação. A forma integral da equação como vimos nas últimas duas etapas, é dita como a ***Formulação Fraca*** do problema.

# Implementação da condição de fronteira de Dirichlet
Como toda função teste $v^\delta(x)$ é zero na fronteira de *Dirichlet*, fica claro que $u^\delta$ deve conter outra função não zero na fronteira, sem isso não seria possível satisfazer a condição de fronteira de *Dirichlet* do problema. Então temos que a solução aproximada $u^\delta$ é uma combinação de outras duas funções: $u^d$, que satisfaz as condições de fronteira de *Dirichlet* e uma função *h*omogênea desconhecida $u^h$, que é zero na fronteira de Dirichelt.
    
 \begin{align}
 u^\delta = \color{Green} {u^d} + \color{Red} {u^h} \\
 u^h(\partial \Omega_D) = 0,\ u^d(\partial \Omega_D) = g_d
 \end{align}


 # Implementação da condição de fronteira de Dirichlet
Da equação dada pela formulação fraca temos:

\begin{equation}
 \int^{1}_{-1} \frac{\partial v}{\partial x}  \frac{\partial u}{\partial x}  \partial x =  v(1)g_N + \int^{1}_{-1}  v f\ \partial x  
\end{equation}
aplicando  $u^\delta = \color{Green} {u^d} + \color{Red} {u^h} $:

 \begin{align}
  & \int^{1}_{-1} \frac{\partial v^\delta}{\partial x}  \frac{\partial u^\delta}{\partial x}  \partial x =  \color{Red} {\int^{1}_{-1} \frac{\partial v^\delta}{\partial x}  \frac{\partial u^h}{\partial x}  \partial x } + \color{Green} { \int^{1}_{-1} \frac{\partial v^\delta}{\partial x}  \frac{\partial u^d}{\partial x}  \partial x} \\
& \color{Red} {\int^{1}_{-1} \frac{\partial v^\delta}{\partial x}  \frac{\partial u^h}{\partial x}  \partial x}= \color{Green} { v^\delta(1)g_N + \int^{1}_{-1}  v^\delta f\ \partial x -    \int^{1}_{-1} \frac{\partial v^\delta}{\partial x}  \frac{\partial u^d}{\partial x}  \partial x }
 \end{align}
 
 a equação  é dada por um sistema algébrico onde os termos do lado direito são conhecidos e a solução homogênea $u^h$ e $v^\delta$ são funções discretizadas. Assim o método de *Galerkin* nos permite reduzir uma *equação diferencial* num problema algébrico.

# Método dos elementos finitos
* Dividimos o domínio $\Omega$ em $\Omega^e$ elementos
* estimamos $v^\delta$ por funções $C^0$ contínuas
* Assim podemos ter derivadas descontinuas nas fronteiras do elemento, apesar da função $C^0$


# Método dos elementos finitos
\begin{align} 
 \int^1_{-1} \frac{\partial v^\delta}{\partial x}\frac{\partial u^\delta}{\partial x} \partial x&= 
\sum^{N_{el}}_{e=1} \int_{\Omega^e}\frac{\partial v^\delta}{\partial x}\frac{\partial u^\delta}{\partial x} \partial x\\
& = - \sum^{N_{el}}_{e=1} \int_{\Omega^e} v^\delta\frac{\partial^2 u^\delta}{\partial x^2} \partial x +\sum^{N_{el}}_{e=1} \left [ v^\delta \frac{\partial u^\delta}{\partial x}\right ]^{\Omega^e_D}_{\Omega^e_E}\\
& = -\int^1_{-1} v^\delta\frac{\partial^2 u^\delta}{\partial x^2}  \partial x +\sum^{N_{el}}_{e=1} \left [ v^\delta \frac{\partial u^\delta}{\partial x}\right ]^{\Omega^e_D}_{\Omega^e_E}
\end{align}

# Método dos elementos finitos

# Exemplo

 Considere a equação unidimensional de Poisson:
\begin{equation}
L(u) \equiv \frac{\partial^2 u}{\partial x^2} + f = 0
\end{equation}
 Onde $f(x)$ é uma função conhecida e as condições de fronteira são:
\begin{equation}
u(0) = g_d= 1,\ \ \frac{\partial u}{\partial x}(1)= g_n = 1
\end{equation}
 Sabendo a formulação fraca, que é dada como:
\begin{equation}
\color{Blue}{\int^1_0 \frac{\partial v^\delta}{\partial x} \frac{\partial u^\delta}{\partial x} \partial x\ } = \color{Purple}{\int^1_0v^\delta f\ \partial x }+ \color{Green}{v^\delta(1)g_n }-\color{Red}{ \int^1_0 \frac{\partial v^\delta}{\partial x} \frac{\partial u^D}{\partial x}\ \partial x}
\end{equation}


# Método dos elementos finitos
\begin{equation}
u^\delta(x) = \sum^2_{i=0} \hat{u_i}\phi_i(x) 
\end{equation} 
<img src="./Figuras/screenshot_elementosfinitos.png" width= 1500>



# Método dos elementos finitos
separação entre a solução *homogênea* e *dirichlet*:
[corrigir uhat]
\begin{align}
 & u^H = \hat{u}_1\phi_1(x) + \hat{u_2}\phi_2(x)
 & u^D = g_D\phi_0(x)
 \end{align}
 
  onde $\hat{u_0}\ e \hat{u_1}$ são arbitrários. A aproximação definida em $u^H$ contém as mesmas funções usadas na função teste $v$, tendo :
 \begin{equation}
 v^\delta(x) = \hat{v_1}\phi_1(x) + \hat{v_2}\phi_2(x)
 \end{equation}
 tanto $\hat{v_0}$ e $\hat{v_1}$ são *desconhecidos*. Definimos $f(x)$ como uma expansão idêntica de $u^\delta$ em relação a $\phi$:
 \begin{equation}
 f(x) = \sum^2_{i=0} \hat{f_0}\phi_0(x) +\hat{f_1}\phi_1(x) + \hat{f_2}\phi_2(x)
 \end{equation}



# Vamos calcular a equação
\begin{equation}
\color{Blue}{\int^1_0 \frac{\partial v^\delta}{\partial x} \frac{\partial u^\delta}{\partial x} \partial x\ } = \color{Purple}{\int^1_0v^\delta f\ \partial x }+ \color{Green}{v^\delta(1)g_n }-\color{Red}{ \int^1_0 \frac{\partial v^\delta}{\partial x} \frac{\partial u^D}{\partial x}\ \partial x}
\end{equation}

 Definidos $u$,$v$,$f$ podemos calcular cada termo na equação:
 primeiro termo:
\begin{align}
\color{Blue}{\int^1_0 \frac{\partial v^\delta}{\partial x} \frac{\partial u^\delta}{\partial x} \partial x\ }& = \color{Blue}{\int^{\frac{1}{2}}_0 (2\hat{v_1})(2\hat{u_1}) \partial x + \int^1_{\frac{1}{2}} (-2\hat{v_1}+2\hat{v_2})(-2\hat{u_1}+2\hat{u_2}) \partial x }
\\
& = \color{Blue}{[\hat{v_1}\ \ \hat{v_2} ]\begin{bmatrix}
4 & -2 \\ 
-2 & 2
\end{bmatrix}
\begin{bmatrix}
\hat{u_1}\ \\ \hat{u_2}\\ 
\end{bmatrix}}
\end{align}

# Método dos elementos finitos

segundo termo:
\begin{align}
\color{Purple}{\int^1_0v^\delta f\ \partial x }= &\color{Purple}{ \int^{\frac{1}{2}}_0 (2\hat{v_1}x)(\hat{f_0}(1-2x) + \hat{f_1}(2x) \partial x} + \\
&\color{Purple}{ \int^1_{\frac{1}{2}} (\hat{v_1}2(1-x) +\hat{v_2}(2x-1))(\hat{f_1}2(1-x) +\hat{f_2}(2x-1)) \partial x}\\
& \color{Purple}{[\hat{v_1}\ \hat{v_2}]\begin{bmatrix}
\frac{1}{12}\hat{f_0} + \frac{1}{3}\hat{f_1} + \frac{1}{12}\hat{f_2} \\ 
\frac{1}{12}\hat{f_0} + \frac{1}{6}\hat{f_2}
\end{bmatrix}}
\end{align}

# Método dos elementos finitos
terceiro termo:
\begin{equation}
\color{Green}{v^\delta(1)g_n = (\hat{v_1}\phi_1(1) + \hat{v_2}\phi_2(1))g_N = [\hat{v_1} \ \hat{v_2}]\begin{bmatrix}
0\\ 
1
\end{bmatrix}g_n}
\end{equation}

# Método dos elementos finitos

quarto termo:
 \begin{align}
 \color{Red}{ \int^1_0 \frac{\partial v^\delta}{\partial x}\frac{\partial u^\delta}{\partial x}} &= \color{Red}{\int^{\frac{1}{2}}_0 (2\hat{v_1})(-2g_D)\ \partial x }\\
             &= \color{Red}{ [\hat{v_1}\ \hat{v_2}]\begin{bmatrix}
-2g_D\\ 
0
\end{bmatrix}}
 \end{align}

# Método dos elementos finitos

Assim substituindo cada termo e botando $[\hat{v_1}\ \hat{v_2}]$ em evidência, temos:

$ \small
\begin{align}
[\hat{v_1}\ \hat{v_2}]\begin{Bmatrix}
\begin{bmatrix}
4 & -2 \\ 
-2 & 2
\end{bmatrix}
\begin{bmatrix}
\hat{u_1}\ \\ \hat{u_2}\\ 
\end{bmatrix}
-
\begin{bmatrix}
\frac{1}{12}\hat{f_0} + \frac{1}{3}\hat{f_1} + \frac{1}{12}\hat{f_2} \\ 
\frac{1}{12}\hat{f_0} + \frac{1}{6}\hat{f_2}
\end{bmatrix}-
\begin{bmatrix}
0\\ 
g_n
\end{bmatrix}+
\begin{bmatrix}
-2g_D\\ 
0
\end{bmatrix}
\end{Bmatrix}
=0
\end{align}
$


 Assim, notamos que para quaisquer $[\hat{v_1},\ \hat{v_2}]$,podemos solucionar essa equação, apenas resolvendo a equação dentro dos **colchetes**,fazendo $g_D$ e $g_N$ ambos iguais a $1$, temos:
\begin{equation}
 \begin{bmatrix}
\hat{u_1}\ \\ \hat{u_2}\\ 
\end{bmatrix}
= \begin{bmatrix}\frac{3}{2} +\frac{1}{24}\hat{f_0}+\frac{5}{24}\hat{f_1}+\frac{1}{8}\hat{f_2} \\ 2 +\frac{1}{24}\hat{f_0}+\frac{1}{4}\hat{f_1}+\frac{5}{24}\hat{f_2} \end{bmatrix}
\end{equation}

# Método dos elementos finitos

Resolvendo esse sistema linear na forma matricial, encontramos a função $u(\bullet)$:
\begin{equation}
u^\delta =
\begin{cases}
1 + x +\frac{x}{12}\hat{f_0} + \frac{5x}{12}\hat{f_1} +\frac{x} {4}\hat{f_2},\ \ 0 \leq x \leq \frac{1}{2}\\
1 + x +\frac{1}{24}\hat{f_0} + \frac{2+x}{12}\hat{f_1} +\frac{1+4x}{24}\hat{f_2},\ \ \frac{1}{2} \leq x \leq 1
\end{cases}
\end{equation}

### Como calcular essas derivadas e integrais para funções mais complexas?

# Método espectral
* Método das diferenças finitas
    * calcula localmente a função
* Método **espectral**
    * calcula globalmente a função
    
tipos de aproximação:
* Modal
    * Fourrier
* Nodal
    * Lagrange
    * Newton

# Método espectral
 Aproximamos uma função $f(x)$ por $P_n(x)$, sabendo $f(x_i)$ para $i = 1,2,...,N$ temos: 
 
\begin{align}
 P_n(x_i) &= f(x_i)\\
 P_n(x) &\equiv \sum_{i = 0}^{N} f(x_i)C_i(x) 
\end{align}



# Método espectral

onde $C_i$ pode ser um polinômio base de Lagrange ou qualquer outro polinômio interpolador.
\begin{equation}
C_i(x) = \prod_{j = 0 \\ j \neq i}^{N} \frac{x - x_j}{x_i - x_j} 
\end{equation} 


# Método espectral

Exemplo para um polinômio de lagrange para 6 pontos:
<img src= "./Figuras/exemplo_polinomio_lagrange.png">

# Porém, essa interpolação nem sempre dá certo. -Carl Runge

<img src= "./Figuras/fenomeno_runge.png" width=850 >


# Fenômeno de Runge
Para minimizar esses erros, temos que achar raízes de polinômios que minimizem o erro de interpolação da função:
## Teorema de cauchy:
\begin{equation}
 f(x) - P_{N+1}(x) = \frac{1}{[N+1]!}f^{(N+1)}(\epsilon){\color{Red}{ \prod^{N}_{i = 0} (x - x_i)}} \\
  \epsilon(x) \in [-1,1].
  \end{equation}



# Fenômeno de Runge

Utilizando as raízes do polinômio $T_i$ de Chebyshev, conseguimos minimizar esse erro:
\begin{align}
    &T_0(x) = 1\\
    &T_1(x) = x\\
    &T_{N+1}(x) = 2xT_N(x) - T_{N-1}(x)\\
    &T_{N}(x) =\cos(n \arccos x)=\cosh(n\,\operatorname{arcosh}\,x)
\end{align}

# Fenômeno de Runge
<img src="./Figuras/chebychev_equidist.png" >

# Método Espectral
## Integração
\begin{align}
\int^{a}_b f(x) \partial x\ \approx \int^{a}_b P_n(x)\ \partial x\ &=\ \sum_{i\ =\ 0}^N f(x_i) \int^{a}_b C_i(x) \partial x\\ &=\ \color{Green}{ \sum_{i\ =\ 0}^N f(x_i) w_i \\}
\end{align} 
onde :
\begin{equation}
 w_i =  \int^{a}_b  C_i(x) \partial x
\end{equation}

# Método Espectral
## Derivação
\begin{equation}
   \frac{\partial f(x)}{\partial x}  \Biggm\lvert_{x=x_i} =\color{Green}{ \sum^{N}_{j\ = 0} f(x_j) C_{ij}}
\end{equation}


 Onde:
 \begin{equation}
  C_{ij} = \frac{\partial C_j(x)}{\partial x} \Biggm\lvert_{x=x_i}
 \end{equation}

# Método Espectral
## Polinômio de Jacobi
*  Polinômios de Jacobi $P^{\alpha,\beta}_n(x)$
    * Família de soluções para problemas de *Sturm-Liouville*
    * polinômios são ***ortogonais*** no intervalo $[-1,1]$, para $\alpha,\beta > -1$
    
Pela fórmula de Rodrigues, essa família é dada por:
\begin{equation}
P_n^{(\alpha,\beta)}(x) = \frac{(-1)^n}{2^n n!} (1-x)^{-\alpha} (1+x)^{-\beta} \frac{d^n}{dx^n} \left\{ (1-x)^\alpha (1+x)^\beta \left (1 - x^2 \right )^n \right\},\\ \text{onde} (\alpha, \beta >-1)
\end{equation}
 sua derivada é dada por:
\begin{equation}
\frac{\partial P^{\alpha,\beta}_n (x)}{\partial x} = \frac{1}{2}(n+\alpha+\beta) P^{\alpha + 1,\beta +1}_{n-1} (x)
\end{equation}


# Método Espectral
## Polinômio de Jacobi
onde podemos encontrar $P^{\alpha + 1,\beta +1}_{n-1}$ pela relação recursiva:
\begin{align}
& P^{\alpha,\beta}_{0} (x) = 1\\
& P^{\alpha,\beta}_{1} (x) = \frac{1}{2}[\alpha - \beta + (\alpha + \beta + 2 )x]\\
& P^{\alpha,\beta}_{n+1} (x)=(a^2_n + a ^3_n x)P^{\alpha,\beta}_{n}(x) - a_n^4P^{\alpha,\beta}_{n-1}(x)  \\
& a^1_n = 2(n+1)(n+ \alpha + \beta + 1)(2n + \alpha +\beta)\\
& a^2_n = (2n + \alpha +\beta + 1)(\alpha^2 - \beta^2)\\
& a^3_n = (2n + \alpha +\beta)(2n + \alpha +\beta + 1)(2n + \alpha + \beta + 2)\\ 
\end{align}


# Método Espectral
## Polinômio de Jacobi
\begin{align}
& \text{Polinômio de Chebychev} (\alpha= \beta = -\frac{1}{2})  \rightarrow  T_n(x) =\frac{2^{2n}(n!)^2}{ (2n)!} P^{(-\frac{1}{2},-\frac{1}{2})}_{n}(x) \\
& \text{Polinômio de Legendre} (\alpha= \beta = 0)  \rightarrow L_n(x) =  P^{(0,0)}_{n}(x)
\end{align}

# Forma Matricial do problema
Ao decidir resolver a equação diferencial, obtemos um sistema de equações diferenciais para cada função teste $v_j^\delta$  e sem perder a igualdade transformamos o problema diferencial em um problema variacional, segue o exemplo abaixo :

\begin{align}
\frac{\partial^2 u^\delta }{\partial x^2} + \lambda u^\delta &= f\\[1.5pt]
v^\delta_j (\frac{\partial^2 u^\delta }{\partial x^2} + \lambda \ u^\delta) \ &= v^\delta_j f\\[1.5pt]
\int_\Omega v^\delta_j (\frac{\partial^2 u^\delta }{\partial x^2} + \lambda v^\delta_j\  u^\delta) \partial x &= \int_\Omega v^\delta_j  f \partial x\\[1.5pt]
\int_\Omega v^\delta_j \frac{\partial^2 u^\delta }{\partial x^2}\ \partial x + \int_\Omega \lambda v^\delta_j\ u^\delta\partial x &=\ \int_\Omega v^\delta_j  f \partial x\\[1.5pt]
\int_\Omega \frac{\partial v^\delta_j }{\partial x} \frac{\partial u^\delta }{\partial x}\ \partial x + \int_\Omega \lambda v^\delta_j\ u^\delta\ \partial x - (v^\delta_j  \frac{\partial u^\delta }{\partial x})\Biggm\lvert_\Omega\ &=\  \int_\Omega v^\delta_j\  f\ \partial x \\ \forall j = 1,2,3,\dots,N 
\end{align}


# Forma Matricial do problema

Como $u^\delta$ é a aproximada de $u$ utilizando a função teste $v^\delta_j$, temos:
\begin{align}
u_i&=u(x_i)\\
u^\delta(x) &= \sum^N_{i=0} v_i^\delta(x)u_i
\end{align}
\begin{align}
\sum^N_{i=1} u_i \int_\Omega \frac{\partial v^\delta_j }{\partial x} \frac{\partial v^\delta_i }{\partial x}\ \partial x +\sum^N_{i=1} u_i  \int_\Omega \lambda\ v^\delta_j v^\delta_i \ \partial x =\  (v^\delta_j  \frac{\partial u^\delta }{\partial x})\Biggm\lvert_\Omega + \int_\Omega v^\delta_j\ f\ \partial x \ \forall j =1,2,\dots,N 
\end{align}  

# Forma Matricial do problema

\begin{align}
\sum^N_{i=1} u_i \left \{  \int_\Omega \frac{\partial v^\delta_j }{\partial x} \frac{\partial v^\delta_i }{\partial x}\ \partial x + \int_\Omega \lambda\ v^\delta_j v^\delta_i \ \partial x \right \} =\  (v^\delta_j  \frac{\partial u^\delta }{\partial x})\Biggm\lvert_\Omega + \int_\Omega v^\delta_j\ f\ \partial x \ \forall j =1,2,\dots,N 
\end{align}  
Que pode ser escrita matricialmente como :
\begin{equation}
\{ M + \lambda S\} U = F
\end{equation}

# Forma Matricial do problema


 Onde chamamos a matriz M de matriz de massa e S de matriz de rigidez.
\begin{align}
M &=
\begin{bmatrix}
\int_\Omega \frac{\partial v^\delta_{1}}{\partial x}\frac{\partial v^\delta_{1}}{\partial x}\ \partial x  & \int_\Omega \frac{\partial v^\delta_{1}}{\partial x}\frac{\partial v^\delta_{2}}{\partial x}\ \partial x  & \dots & \int_\Omega \frac{\partial v^\delta_{1}}{\partial x}\frac{\partial v^\delta_{N}}{\partial x}\ \partial x  \\ 
\int_\Omega \frac{\partial v^\delta_{1}}{\partial x}\frac{\partial v^\delta_{2}}{\partial x}\ \partial x  & \int_\Omega \frac{\partial v^\delta_{2}}{\partial x}\frac{\partial v^\delta_{2}}{\partial x}\ \partial x  & \dots & \int_\Omega \frac{\partial v^\delta_{2}}{\partial x}\frac{\partial v^\delta_{N}}{\partial x}\ \partial x  \\ 
\vdots & \vdots & \vdots & \dots \\
\int_\Omega \frac{\partial v^\delta_{N}}{\partial x}\frac{\partial v^\delta_{1}}{\partial x}\ \partial x  & \int_\Omega \frac{\partial v^\delta_{N}}{\partial x}\frac{\partial v^\delta_{2}}{\partial x}\ \partial x  & \dots & \int_\Omega \frac{\partial v^\delta_{N}}{\partial x}\frac{\partial v^\delta_{N}}{\partial x}\ \partial x  \\ 
\end{bmatrix}, \\ \\
S &= \begin{bmatrix}
\int_\Omega v^\delta_1 v^\delta_1 \partial x & \int_\Omega v^\delta_1 v^\delta_2 \partial x & \dots & \int_\Omega v^\delta_1 v^\delta_N \partial x \\ 
\int_\Omega v^\delta_2 v^\delta_1 \partial x & \int_\Omega v^\delta_2 v^\delta_2 \partial x & \dots & \int_\Omega v^\delta_2 v^\delta_N \partial x\\ 
\vdots & \vdots & \ddots & \vdots \\
\int_\Omega v^\delta_N v^\delta_1 \partial x & \int_\Omega v^\delta_N v^\delta_2 \partial x & \dots & \int_\Omega v^\delta_N v^\delta_3 \partial x
\end{bmatrix},\
U = \begin{bmatrix}
u_1\\ 
u_2\\ 
\vdots \\
u_N
\end{bmatrix}\\
\end{align}
onde M é a matriz de massa e S é a matriz de rigidez (stifness)

# Forma Matricial do problema

\begin{align}
F &= \begin{bmatrix}
\int_\Omega v^\delta_1\ f\ \partial x +  (v^\delta_1  \frac{\partial u^\delta }{\partial x})\Biggm\lvert_\Omega \\ 
\int_\Omega v^\delta_2\ f\ \partial x +  (v^\delta_2  \frac{\partial u^\delta }{\partial x})\Biggm\lvert_\Omega \\ 
\vdots \\
\int_\Omega v^\delta_N\ f\ \partial x +  (v^\delta_N  \frac{\partial u^\delta }{\partial x})\Biggm\lvert_\Omega\\ 
\end{bmatrix}
\end{align}

# Mapeamento

Quando tratamos das funções de bases, temos as funções globais e as locais. Apesar das funções globais serem na teoria usadas para o calcular as matrizes de *massa* e de *rigidez*, na prática decompomos as funções globais em elementos e utilizamos dessa decomposição as funções de base locais para o cálculo das matrizes.
## Exemplo
 Tomemos como exemplo as funções lineares globais $\Phi$ em um domínio $\Omega = \{x\ |\ 0\leq x \leq 1\}$ subdividido em **3** elementos. Temos 4 graus de liberdade nessa expansão, definida no máximo em 2 elementos ao mesmo tempo,que vale 1 em um dos extremos desses elementos e decresce para 0 para o em direção aos outros extremos.

# Mapeamento
\begin{equation}
\Phi_1(x) = \left\{\begin{matrix}
1 - 3x,\ 0 \leq x \leq \frac{1}{3}\\
0\ , \frac{1}{3} < x \leq 1
\end{matrix}\right.\ \ 
\Phi_2(x) = \left\{\begin{matrix}
3x,\ 0 \leq x \leq \frac{1}{3}\\
2 - 3x\ , \frac{1}{3} < x \leq \frac{2}{3}\\
0\ , \frac{2}{3} < x \leq 1
\end{matrix}\right.\\
\end{equation} 
\begin{equation}
\Phi_3(x) = \left\{\begin{matrix}
0 ,\ 0 \leq x < \frac{1}{3}\\
3x - 1\ ,\ \frac{1}{3} < x \leq \frac{2}{3}\\
-3x + 3 \ ,\ \frac{2}{3} < x \leq 1
\end{matrix}\right.\\
\Phi_4(x) = \left\{\begin{matrix}
0\ ,\ \ 0 \leq x < \frac{1}{3}\\
0\ ,\ \frac{1}{3} < x \leq \frac{2}{3}\\
3x -2 \ ,\ \frac{2}{3} < x \leq 1
\end{matrix}\right.
\end{equation}

# Mapeamento
<img src="./Figuras/Matrix_elem_global.png">

# Mapeamento
Também podemos definir funções locais $\phi^e_p(x)$ onde $\Omega_e = \{x \| x_{e1} \leq x \leq x_{e2} \}$:
\begin{equation}
\phi^e_1\ = \left\{\begin{matrix}
\frac{x_{e1}\ -\ x}{x_{e2} - x_{e1}}, x \in \Omega_e \\
0 ,\ x \notin \Omega_e 
\end{matrix}\right.
\phi^e_2\ = \left\{\begin{matrix}
\frac{x\ -\ x_{e1}}{x_{e2} - x_{e1}}, x \in \Omega_e \\
0 ,\ x \notin \Omega_e 
\end{matrix}\right.
\end{equation} 

# Mapeamento

<img src="./Figuras/Matrix_element_local.png">
\begin{align}
\Phi_1 &= \phi_{1}^{1} \\[0.5pt]
\Phi_2 &= \phi_{2}^{1} + \phi_{1}^{2} \\[0.5pt]
\Phi_3 &= \phi_{2}^{2} + \phi_{1}^{3} \\[0.5pt]
\Phi_4 &= \phi_{2}^{3}
\end{align}

# Mapeamento

\begin{equation}
\begin{bmatrix}
\phi_1^1\\[1.9pt] 
\phi_2^1\\[1.9pt]
\phi_1^2\\[1.9pt] 
\phi_2^2\\[1.9pt]
\phi_1^3\\[1.9pt] 
\phi_2^3\\[1.9pt] 
\end{bmatrix}\ = \
\begin{bmatrix}
1 &0  & 0 & 0 \\ 
0 & 1 & 0 & 0\\ 
0 & 1 & 0 & 0\\ 
0 &0  & 1 & 0\\ 
0 &0  & 1 & 0\\ 
0 &0  & 0 & 1
\end{bmatrix}\
\begin{bmatrix}
\Phi_1\\ 
\Phi_2\\ 
\Phi_3\\ 
\Phi_4
\end{bmatrix}
\end{equation}


# Mapeamento 

Devido ao fato da matriz $A$ ser uma matriz esparsa e portanto numericamente ineficiente para ser usada como um operador, utilizamos outra saída. Essa alternativa é uma matriz **map**eadora em que cada coluna é um elemento (e) e cada linha o indice da função global (i).
\begin{equation}
map[1,i]= \begin{bmatrix}
1\\
2
\end{bmatrix},\
map[2,i]= \begin{bmatrix}
2\\
3
\end{bmatrix}\
,map[3,i]= \begin{bmatrix}
3\\
4
\end{bmatrix}
\end{equation}
usando um pseudo código, utilizamos essa matriz mapeadora para montar a função global dada a local:
<img src= "./Figuras/pseudocodigo.png" >

# Mapeamento 
Assim a matriz de massa *global* $M$ definida como $M_{ij} = \int_\Omega \Phi_i\ \Phi_j \partial x$ analogamente como  o código anterior, pode ser reescrita a função de uma matriz de massa $M^e$ *local*, utilizando uma matriz de mapeamento $map$ similar a anterior.
<img src="./Figuras/Matrix_element.png">

* podemos usar polinômios mais complexos [Lagrange, Legendre,...]

# Condensação estática
 Após o mapeamento das matrizes temos então a tarefa de resolver o sistema linear.Utilizando o fato do mapeamento ter separado as funções de base para os modos ***internos*** (i) e os modos de ***fronteira*** (b) de cada elemento, aplicamos a técnica de **condensação estática**. Onde com remapeando os vetores x e f do sistema linear $A\  x = f$ de maneira a ficar primeiro os coeficientes de fronteira e depois os coeficientes internos temos o sistema reescrito como:
  \begin{equation}
    x = \left\{\begin{matrix} x_b \\ x_i \\ \end{matrix}\right\}\ ,\ f =  \left\{\begin{matrix} f_b \\ f_i \\ \end{matrix}\right\}\
  \end{equation}



# Condensação estática
E com o remapeamento feito anteriormente, temos a matriz A global como:

<img src="./Figuras/Matriz_global.png">
onde cada $A_{\cdot \cdot}$ é uma sub matriz de A em que $A_{ii}$ é uma matriz em banda diagonal em bloco, $A_{ib}$ e $A_{bi}$ são  matrizes esparsas e $A_{bb}$ é uma matriz em bloco. [Assim, podemos resolve-la localmente]



# Condensação estática
\begin{equation}
A\cdot x  = \left[ 
\begin{matrix} 
A_{bb} & A_{bi} \\
A_{ib} & A_{ii} \\
\end{matrix}\right] \cdot \left\{
\begin{matrix} x_b \\ x_i \\ 
\end{matrix}\right\} = 
\left\{\begin{matrix} f_b \\ f_i \\\end{matrix}\right\} 
\end{equation}

# Condensação estática

Para resolver esse sistemas fazemos:
\begin{equation}
\color{Red}{x_i} = A_{ii}^{-1} f_i - A_{ii}^{-1} A_{ib} \color{Green}{x_b}
\end{equation}
Que substituindo $\color{Red}{x_i}$ na equação de  $\color{Green}{x_b}$, obtemos:
\begin{equation}
\left( A_{bb} - A_{bi}A_{ii}^{-1} A_{ib} \right) \color{Green}{x_b} = f_b - A_{bi}A_{ii}^{-1} f_i\\
A'_{bb} \cdot \color{Green}{x_b} = f'_b
\end{equation}
onde:
\begin{align}
& A'_{bb} = A_{bb} - A_{bi}A_{ii}^{-1} A_{ib}\\ 
& f'_b = f_b - A_{bi}A_{ii}^{-1} f_i
\end{align}
Assim, utilizando o remapeamento e a condensação estática, encontramos $\color{Green}{x_b}$ e $\color{Red}{x_i}$ da equação.

* na implementação do código $A_{bb}$ pode ter outros formatos [em banda, esparça genérica]



# Simulação

## Convergência do erro no fenômeno Runge
Vamos agora ver a convergência do erro da aproximaçãode uma função de runge $\frac{1}{1+x^2},\ x\ \in [-5,5]$, para pontos **equidistantes** e pontos igualmente espaçados, utilizando o polinômio de **Lagrange**.
<img src="./Figuras/interpolacao_todas.png">


# Simulação
## Convergência do erro no fenômeno Runge [método espectral]
<img src="./Figuras/glj_equi.png">

# Simulação
## Convergência do erro no fenômeno Runge [método espectral]
<img src="./Figuras/glj_cheb.png" width=750>

Apesar de bem próximos, notamos que usando os pontos de Chebyshev converge um pouco melhor segundo o teorema anterior

## Convergência do erro no fenômeno Runge [método HP]
<img src="./Figuras/interp_usando_FEM.png">
<img src="./Figuras/interp_usando_FEMfixo.png">

<img src="./Figuras/convergencia_erro_FEM2.png">

# Método HP na resolução de equações diferenciais
## Condição de Dirichlet
 Agora, resolveremos uma equação diferencial de segundo grau. Para tanto, irei apresentar uma equação com solução conhecida ($sin(2\ k \pi x)$).
 \begin{align}
y'' + y &= (1 + 4 (k \pi)^2)sin(2 k \pi x) \\
y(-1) &= sin(-2\ k \ \pi ) = 0 ,\ y(1) = sin(2\ k\ \pi) = 0 \ \ \forall k = 1,2,\dots \\
\end{align}	
<img src="./Figuras/solu_edo_simul.png">

## Convergência do erro para Dirichlet
<img src="./Figuras/convergencia_erro_HP.png">

# Condição de Neumann
 Agora utilizamos o método HP para um problema no qual temos em uma das extremidades uma condição de ***Neumann***. Nesse exemplo usarei como solução do problema $y(x) = \cos(10 x) \sin(25 x)$, onde $ x \in [-1,1]$ e a condição de Neuman está em $x=1$.
\begin{equation}
\\-y'' + y = 726\,\cos \left(10\,x\right)\,\sin \left(25\,x\right)+500\,\sin 
 \left(10\,x\right)\,\cos \left(25\,x\right) 
\end{equation}
<img src="./Figuras/Neumm_10_15.png" width= 650>

A convergência dos erros utilizando o método P e o método H continuam semelhantes à convergência do erro dos métodos aplicado a condição de ***Dirichlet***:
<img src="./Figuras/convergencia_erro_Neumm.png">

 Agora veremos a convergência do erro para a aproximada da *derivada* da solução no ponto onde temos a condição de ***Neumann*** ocorre 
 <img src="./Figuras/erro_derivada.png">
* [Não obedece exatamente a condição de neumman mas a solução converge para isso ]

[ o static condensation pode ser usado para problemas multidimensionais como está implementado]

# Onde o método se sobrepõe ?


<img src="./Figuras/Neumm_1_50.png">

<img src="./Figuras/Neumm_2_20.png">

<img src="./Figuras/Neumm_4_20.png">

# Conclusões

* Abrangência do trabalho
* Dificuldades da Teoria x Prática
* Nova técnica para resolução diferencial
* Blas e Lapack
* Implementação de código em equipe utilizando Github

# Fim